# GENERATE PARKING DEMAND

This module takes the arrival and departure demand along with the mode choice percentage and extracts the appropriate amount of demand from the people demand and converts it to cars for parking. 

```python
def create_sumo_demand_parking(people,
              percentOfParking=.35,
              stops=stop_dict = {
                "A":['A_bot_1','A_bot_2','A_bot_3'],
                "B":['B_bot_1','B_bot_2','B_bot_3'],
                "C":['C_bot_1','C_bot_2','C_bot_3'],
                "D":['D_depart_1','D_depart_2','D_arrive_1','D_arrive_2','D_service'],
                "E":['E_bot_1','E_bot_2','E_bot_3'],

                },
             end_weight = [.2,.8],
             start_weights = [.225,.225,.275,.275],
             stop_duration = np.random.exponential(20,1000) + np.random.normal(60,5,1000),
                                         ):
```

> - **people:** *DataFrame* - is the people demand for the 
> - **percentOfLimo** *Float* - perecent of passengers going to curb
> - **stops** *Dictionary* - informs the stops for the passenger vehicles
> - **end_weight** *List* - list of floats that add to 1. Informs the destination of each trip. 
> - **start_weights** *List or Array* - list of floats that add to 1. Informs the origin of each trip.
> - **stop_duration** *List or Array* - distribution of dwell times on the curb

```python
def add_buses_for_people(buses,
                         capacity=50,
                         starts = {
                                "A":['A_bot_1','A_bot_2','A_bot_3'],
                                "B":['B_bot_1','B_bot_2','B_bot_3'],
                                "C":['C_bot_1','C_bot_2','C_bot_3'],
                                "D":['D_depart_1','D_service'],
                                "E":['E_bot_1','E_bot_2','E_bot_3']},
                         stops = ['park_south_term_1','park_south_emp_3',
                                  'park_north_emp_1'],
                         stop_duration=np.random.exponential(20,10000) + np.random.normal(60,5,10000)
                        ): 
```

> - **buses** *Dictionary* - output dictionary from generate parking trips
> - **capacity** *Interger* - number of people each bus hold for capacity
> - **starts** *Dictionary* - set dictionary that holds the starting routes of the buses
> - **stops** *List* - list of stops the buses will be going to for parking
> - **stop_duration** *list/array* - distribution of dwell times at curb to sample from

In [82]:
from __future__ import print_function
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment
from lxml import etree
from copy import copy
import os
import sys
import inspect
from xml.dom import minidom
import pandas as pd
import numpy as np
import random
sys.path.insert(0, os.path.abspath('../Generate_SUMO_demand/'))
import DFW_gen_flow as gf

In [83]:
Date = '2018-5-13' 

In [84]:
folder = "../Example_Files/Demand_CSV"
selected = pd.read_csv("selected_demand.csv")
level = selected.loc[selected['Date'] == Date]['Volume_Category'].item()
file_name = Date + '.' + level +".people.csv"
people = pd.read_csv(os.path.join(folder,file_name),index_col=[0])

people.head()

,Arrive_A_people,Arrive_B_people,Arrive_C_people,Arrive_D_people,Arrive_E_people,Depart_A_people,Depart_B_people,Depart_C_people,Depart_D_people,Depart_E_people,Depart_total,Arrival_total,pass_thru,Total
0,120,65,126,65,12,120,67,128,65,12,392,388,44.0,780
1,70,39,77,31,7,70,41,79,31,7,228,224,24.0,452
2,50,29,56,26,5,51,29,58,26,5,169,166,32.0,335
3,58,34,63,31,5,58,34,65,31,5,193,191,16.0,384
4,31,19,34,19,5,32,19,36,19,7,113,108,32.0,221


In [85]:
def create_sumo_demand_parking(people,
                                      level,
                                      percentOfParking=.25,
                                      peopleToCars=1.7,
                                      starts= {
                                        'parking' : ['park_south_term_1','park_south_term_2',
                                                 'park_south_term_3','park_south_term_4',
                                                 'park_south_emp_1','park_south_emp_2',
                                                 'park_south_emp_3','park_north_emp_1'],
                                        "A":['A_park_enter'] ,
                                        "B":['B_park_enter_1'] ,
                                        "C":['C_park_enter_1'] ,
                                        "D":['D_park'] ,
                                        "E":['E_park_enter'],

                                        },
                                     arrive = {
                                        'parking' : ['park_south_term_1','park_south_term_2',
                                                 'park_south_term_3','park_south_term_4',
                                                 'park_south_emp_1','park_south_emp_2',
                                                 'park_south_emp_3','park_north_emp_1'],
                                        "A":['A_park_exit'] ,
                                        "B":['B_park_exit'] ,
                                        "C":['C_park_exit_1'] ,
                                        "D":['D_park'] ,
                                        "E":['E_park_exit_1'],

                                        },
                                         end_weight = [.2,.8],
                                         start_weights = [.225,.225,.275,.275],
                                         stop_duration = np.random.exponential(20,10000) + np.random.normal(60,5,10000),
                                         ):
    
    start_depart = ['South_1', 'South_Plaza', 'North_Plaza', 'North_1']
    end_arrive = ['South_Exit', 'North_Exit']
    columns = ['Arrive_A_people','Arrive_B_people','Arrive_C_people','Arrive_D_people',
               'Arrive_E_people','Depart_A_people','Depart_B_people',
               'Depart_C_people','Depart_D_people','Depart_E_people']
    
    routes = Element('routes')
    routes.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    routes.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')
    people['seconds'] = np.array(people.index) * 30 * 60
    count = 1
    buses = {}
    for column in columns:
        column_string = column.split('_')
        terminal = column_string[1]
        for t,numberOfPeople in enumerate(people[column]):
            numberOfVehicles = round((numberOfPeople/peopleToCars) * percentOfParking)
            for i in range(numberOfVehicles):
                time = people['seconds'][t] + round(np.random.uniform(0,1800))
                if column_string[0] == 'Arrive':
                    begin = arrive[terminal] + arrive['parking']
                    start = np.random.choice(begin)
                    end = np.random.choice(end_arrive)
                    #add buses from terminal to parking
                    
                else:
                    finish = starts[terminal] + starts['parking']
                    start = np.random.choice(start_depart,p=start_weights)
                    end = np.random.choice(finish)
                    #add buses from parking to terminal
                
                halfHour = round(time/3600)
                try:
                    buses[terminal + '_' + str(halfHour)] += 1
                except:
                    buses[terminal + '_' + str(halfHour)] = 1
                trip = Element('trip')
                trip.set('id', column + '_parking_' + str(count))
                trip.set('type', 'passenger')
                trip.set('color', "#bb0000")
                trip.set('depart',str(time))
                trip.set('from',start)
                trip.set('to',end)
                trip.set('departSpeed', "max")
                trip.set('departLane', "best")
                
                count+=1
                routes.append(trip)
                

    routes[:] = sorted(routes, key=lambda child: (child.tag,float(child.get('depart'))))

    file_name = Date + "." + level+ ".parking.xml"
    folder = "../Example_Files/TempDemandXML"
    print("Saving to xml: ", file_name)
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(routes)).toprettyxml(encoding="utf-8"))
    
    return buses

In [31]:
buses = create_sumo_demand_limo_curbside(people,'High',.25)

Saving to xml:  2018-5-13.High.parking.xml


In [77]:
buses

{'A_0.0': 36,
 'A_1.0': 35,
 'A_2.0': 28,
 'A_3.0': 38,
 'A_4.0': 142,
 'A_5.0': 252,
 'A_6.0': 286,
 'A_7.0': 238,
 'A_8.0': 323,
 'A_9.0': 400,
 'A_10.0': 362,
 'A_11.0': 386,
 'A_12.0': 356,
 'A_13.0': 477,
 'A_14.0': 492,
 'A_15.0': 475,
 'A_16.0': 506,
 'A_17.0': 574,
 'A_18.0': 453,
 'A_19.0': 456,
 'A_20.0': 348,
 'A_21.0': 342,
 'A_22.0': 249,
 'A_23.0': 202,
 'A_24.0': 77,
 'B_0.0': 20,
 'B_1.0': 20,
 'B_2.0': 16,
 'B_3.0': 27,
 'B_4.0': 94,
 'B_5.0': 166,
 'B_6.0': 182,
 'B_7.0': 146,
 'B_8.0': 180,
 'B_9.0': 225,
 'B_10.0': 218,
 'B_11.0': 234,
 'B_12.0': 226,
 'B_13.0': 272,
 'B_14.0': 274,
 'B_15.0': 286,
 'B_16.0': 302,
 'B_17.0': 316,
 'B_18.0': 258,
 'B_19.0': 254,
 'B_20.0': 184,
 'B_21.0': 176,
 'B_22.0': 134,
 'B_23.0': 106,
 'B_24.0': 46,
 'C_0.0': 38,
 'C_1.0': 40,
 'C_2.0': 29,
 'C_3.0': 43,
 'C_4.0': 152,
 'C_5.0': 294,
 'C_6.0': 308,
 'C_7.0': 270,
 'C_8.0': 352,
 'C_9.0': 432,
 'C_10.0': 380,
 'C_11.0': 456,
 'C_12.0': 399,
 'C_13.0': 512,
 'C_14.0': 524,
 'C_1

In [75]:
def add_buses_for_people(buses,
                         capacity=50,
                         starts = {
                                "A":['A_bot_1','A_bot_2','A_bot_3'],
                                "B":['B_bot_1','B_bot_2','B_bot_3'],
                                "C":['C_bot_1','C_bot_2','C_bot_3'],
                                "D":['D_depart_1','D_service'],
                                "E":['E_bot_1','E_bot_2','E_bot_3']},
                         stops = ['park_south_term_1','park_south_emp_3',
                                  'park_north_emp_1'],
                         stop_duration=np.random.exponential(20,10000) + np.random.normal(60,5,10000)
                        ): 
    routes = Element('routes')
    routes.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    routes.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')
    count = 0
    for key in buses:
        key_string = key.split('_')
        terminal = key_string[0]
        hour = key_string[1]
        hr = float(hour) * 3600
        passengers = buses[key]
        number_of_buses = int(np.ceil(passengers/capacity)) - 4
        if number_of_buses > 0:
            
            for i in range(number_of_buses):
                time = hr + round(np.random.uniform(0,3600))
                start = np.random.choice(starts[terminal])
                end = start
                trip = Element('trip')
                trip.set('id', terminal + '_' + str(count))
                trip.set('type', 'bus')
                trip.set('color', "#bb0000")
                trip.set('depart',str(time))
                trip.set('from',start)
                trip.set('to',end)
                trip.set('departSpeed', "max")
                trip.set('departLane', "best")

                count+=1
                routes.append(trip)

                count+=1
                for stop in stops:  
                    duration = str(round(np.random.choice(stop_duration)))
                    ET.SubElement(trip,"stop",busStop=stop,duration=duration,parking='true')



    routes[:] = sorted(routes, key=lambda child: (child.tag,float(child.get('depart'))))

    file_name = Date + "." + level+ ".MoreBuses.xml"
    folder = "../Example_Files/TempDemandXML"
    print("Saving to xml: ", file_name)
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(routes)).toprettyxml(encoding="utf-8"))
    
    

In [76]:
add_buses_for_people(buses,25)

Saving to xml:  2018-5-13.high.MoreBuses.xml
